# Plot RMS changes 

This notebook will be plotting the differences in RMS due to a synthetic magma body at depth

In [1]:
from pathlib import Path
from mtpy import MTData, MTCollection
from mtpy.modeling.plots.plot_modem_rms import PlotRMS
from mtpy.modeling.modem.data import Data
import numpy as np

In [2]:
h5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests.h5")

In [4]:
survey_names = [
    "data",
    "inv_z03_t02_c02_040",
    "doi_25km",
    "doi_30km",
    "doi_35km",
    "doi_40km",
    "doi_45km",
    "doi_55km",
    "doi_70km",
    "doi_80km",
    "lp_melt",
]

In [6]:
md_dict = {}
with MTCollection() as mc:
    mc.open_collection(h5_path)
    for survey_id in survey_names:
        mc.working_dataframe = mc.master_dataframe[mc.master_dataframe.survey==survey_id]
        md_dict[survey_id] = mc.to_mt_data().to_dataframe()

25:05:15T13:31:51 | INFO | line:769 |mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests.h5


In [4]:
def calculate_rms(data_df, model_df):
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        with np.errstate(divide="ignore", invalid="ignore"):
            model_df[f"rms_{col.replace('_', '')}"] = np.abs(
                data_df[col] - model_df[col]
            ) / (np.real(data_df[f"{col}_model_error"]) * np.sqrt(2))
    return model_df

In [3]:
def calculate_rms_change(model_01_df, model_02_df):
    """ 
    Assumes 1 is the original, 2 is the new and returns model 2 as 2 - 1

    In that sense if the value is positive then 2 is larger rms and if
    the value is negative then 2 got smaller.
    """
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        key = f"rms_{col.replace('_', '')}"
        with np.errstate(divide="ignore", invalid="ignore"):
            model_02_df[key] = (model_02_df[key] - model_01_df[key]) / model_02_df[key]

    return model_02_df

In [5]:
data_obj = Data()
data_df = data_obj.read_data_file(Path(
    r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_modem_data_z03_t02_tec_07.dat"
))

In [6]:
model_data_obj = Data()
model_data_df = data_obj.read_data_file(
    Path(
        r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_test_doi_80km.dat"
    )
)

test_data_obj = Data()
test_data_df = data_obj.read_data_file(
    Path(
        r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_outside_test.dat"
    )
)

In [7]:
sorted(md.survey_ids)

['17km_10ohmm',
 '17km_20ohmm',
 '17km_30ohmm',
 '17km_3ohmm',
 '60km_10ohmm',
 '60km_30ohmm',
 '60km_3ohmm',
 'data',
 'gpc_10ohmm',
 'gpc_20ohmm',
 'gpc_30ohmm',
 'gpc_3ohmm',
 'inv_z03_t02_c02_040',
 'inv_z03_t02_c02x2_028',
 'li17km_10ohmm',
 'li17km_30ohmm',
 'li17km_3ohmm']

In [8]:
md_dict = {}
for survey in sorted(md.survey_ids):
    md_dict[survey] = md.get_survey(survey).to_mt_dataframe()

In [18]:
md_dict["data"] = data_df

In [7]:
base_model_01 = calculate_rms(data_df.dataframe, model_data_df.dataframe)
# base_model_02 = calculate_rms(md_dict["data"].dataframe, md_dict["inv_z03_t02_c02x2_028"].dataframe)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
import matplotlib.ticker as ticker

# Create a segmented colormap with white band in the middle
def create_segmented_bwr_cmap():
    # Define colors for segments (blue to white to red)
    blues = plt.cm.Blues(np.linspace(0.3, 1, 5))  # 5 shades of blue
    whites = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])  # 2 white segments for -0.1 to 0.1
    reds = plt.cm.Reds(np.linspace(0, 0.7, 5))  # 5 shades of red
    
    # Combine the colors
    colors_array = np.vstack([blues[::-1], whites, reds])
    
    # Create nodes for the segments (ensure we have white at -0.1 to 0.1)
    nodes = np.concatenate([
        np.linspace(-1, -0.1, 6),  # 5 blue segments
        np.linspace(-0.1, 0.1, 2),  # white segment
        np.linspace(0.1, 1, 6)      # 5 red segments
    ])
    
    # Create the colormap
    # cmap = colors.LinearSegmentedColormap.from_list('SegmentedBWR', list(zip(nodes, colors_array)))
    cmap = colors.LinearSegmentedColormap.from_list('SegmentedBWR', list(colors_array))
    return cmap

In [9]:
rms_cmap = create_segmented_bwr_cmap()

In [10]:
test_df = calculate_rms(data_df.dataframe, test_data_df.dataframe)
delta_model_df = calculate_rms_change(base_model_01, test_df)

In [11]:
dp = PlotRMS(delta_model_df)
dp.rms_min = -.1
dp.rms_max = .08
dp.rms_step = .02
dp.fig_num = 4
dp.fig_size = [20, 10]
#dp.rms_cmap = "mt_rd2wh2bl"
dp.rms_cmap = rms_cmap
dp.plot_station = False
dp.box_size = 20
dp.dx = .0175

dp.plot()
dp.save_plot(h5_path.parent.joinpath(f"{'data'}_vs_{'ouside_purged'}_rms.png"), fig_dpi=300)

25:05:21T15:21:40 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))
25:05:21T15:21:43 | INFO | line:125 |mtpy.imaging.mtplot_tools.base | save_plot | Saved figure to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\data_vs_ouside_purged_rms.png


In [12]:
for key in md_dict.keys():
    if key in ["data", 'inv_z03_t02_c02_040']:
        continue
    try:
        model_df = calculate_rms(md_dict["data"].dataframe, md_dict[key])
    except ValueError:
        print(f"Skipping {key}")
        continue
    for base_df, base_id in zip([base_model_01], [40]):
        delta_model_df = calculate_rms_change(base_df, model_df)

        dp = PlotRMS(delta_model_df)
        dp.rms_min = -.1
        dp.rms_max = .08
        dp.rms_step = .02
        dp.fig_num = 4
        dp.fig_size = [20, 10]
        #dp.rms_cmap = "mt_rd2wh2bl"
        dp.rms_cmap = rms_cmap
        dp.plot_station = False
        dp.box_size = 20
        dp.dx = .0175

        dp.plot()
        dp.save_plot(h5_path.parent.joinpath(f"{key}_vs_{base_id}_delta_rms.png"), fig_dpi=300)

NameError: name 'md_dict' is not defined

In [13]:
for key in md_dict.keys():
    if key in ["data", 'inv_z03_t02_c02_040']:
        continue
    try:
        model_df = calculate_rms(md_dict["data"].dataframe, md_dict[key])
    except ValueError:
        print(f"Skipping {key}")
        continue
    for base_df, base_id in zip([base_model_01], [40]):
        delta_model_df = calculate_rms_change(base_df, model_df)
        print(f"Mean RMS by component: {key}")
        print(f"{delta_model_df[['rms_zxx', 'rms_zyy', 'rms_zxy', 'rms_zyx', 'rms_tzx', 'rms_tzy']].mean()}")
        print("\n")
        print(f"Mean RMS: {delta_model_df[['rms_zxx', 'rms_zyy', 'rms_zxy', 'rms_zyx', 'rms_tzx', 'rms_tzy']].mean().mean()}")
        print("="*20)

NameError: name 'md_dict' is not defined

In [44]:
key

'lp_melt'

In [47]:
from mtpy.modeling import StructuredGrid3D

In [53]:
model_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo")
mfn_list = (
    list(model_path.glob("cl_gpc*.rho"))
    + list(model_path.glob("*cl_li_17km*.rho"))
    + [model_path.joinpath("cl_test_lp_melt.rho")]
)

In [54]:
mfn_list

[WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_10_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_20_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_30_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_gpc_magma_test_3_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_10_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_30_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_li_17km_magma_test_3_ohmm.rho'),
 WindowsPath('c:/Users/jpeacock/OneDrive - DOI/ClearLake/modem_inv/inv_05_topo/cl_test_lp_melt.rho')]

In [55]:
s1 = StructuredGrid3D()
s1.from_modem(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_z03_t02_c02_040.rho")

In [56]:
for mfn in mfn_list:
    s2 = StructuredGrid3D()
    s2.from_modem(mfn)
    s1.res_model *= s2.res_model

s1.res_model = s1.res_model **(1/(len(mfn_list) + 1))
s1.to_modem(model_fn=model_path.joinpath("cl_sm_test_avg.rho"))

25:05:15T16:11:23 | INFO | line:873 |mtpy.modeling.structured_mesh_3d | to_modem | Wrote file to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_sm_test_avg.rho
